# INSTALL

See [documentation to set-up the local OSRM server](https://github.com/satellogic/open-impact/blob/master/gambia/routing-server.md)


# Travel time

This notebook shows how to calculate traveling time over known roads in different modes of transportation (bike, car, truck, walking, ...).


In [4]:
osrm_server="http://0.0.0.0:5000"
osrm_server="https://router.project-osrm.org" #IF you want to test the code running a remote server

## Simple OD pair call

In [5]:
import requests  #http framework to make Mapbox API requests for routes
import json # handle response as json
import datetime # save timestamp
 
url=osrm_server+"/route/v1/driving/"
comma="%2C"
sep="%3B"
origin=[14.721761,-17.418823]
destination=[13.480448,-13.958130]
fullurl=url+str(origin[1])+','+str(origin[0])+";"+str(destination[1])+','+str(destination[0])
response = requests.get(fullurl) #do the request
response.raise_for_status() # ensure we notice bad responses
print(fullurl)
# http://localhost:5000/route/v1/driving/-17.418823,14.721761;-13.95813,13.480448'
print(json.loads(response.text)['routes'][0]['distance']," meters")
# 510247, ' meters'
print (json.loads(response.text)['routes'][0]['duration']," seconds")
# 30281.4, ' seconds'



https://router.project-osrm.org/route/v1/driving/-17.418823,14.721761;-13.95813,13.480448
503214.2  meters
55963.9  seconds


## Matrix call

In [6]:
import json

# Read Origins
origin_file='./villages.geojson'

with open(origin_file) as f:
    villages = json.load(f)
print("Read %i villages" % len(villages['features']) )

# Read Destinations
destination_file='./health.geojson'

with open(destination_file) as f:
    health = json.load(f)
print("Read %i health facilities" % len(health['features']) )

Read 798 villages
Read 28 health facilities


In [8]:
import csv
import requests
import json
import numpy as np

#calculate all pairs of distance
#print [time,distance] in [hours,km] to get for one point to the other
url=osrm_server+"/table/v1/driving/"


#Example from API 
#curl 'http://router.project-osrm.org/table/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219'

server=osrm_server+"/table/v1/driving/"        
comma="%2C"
Sep="%3B"

batch=100
origin_type='farmer'
origins['t_'+origin_type]=-1
origins['m_'+origin_type]=-1

#origins is a panda DataFrame as the set of origins for the routes
#destinations is a panda DataFrame as the set of destinations for the routes.

for i in np.arange(origins.shape[0]/batch):
    print("Doing batch %i, [%i,%i] of %i"
          %(i,batch*i,batch*(i+1),origins.shape[0]),end="\r")
    origins_batch=origins.iloc[int(batch*i):].head(n=batch)
    
    destinations=";".join([str(i[1])+','+str(i[0]) for i in destinations[['lat','lon']].values])
    origins_ids=origins_batch['ids']
    
    trail=".json?destinations="+\
    ';'.join([str(x) for x in np.arange(len(destinations))])+\
    "&sources="+\
    ';'.join([str(x) for x in np.arange(len(destinations),len(destinations)+len(destinations_batch))])
    
    fullurl= server+destinations_loc+";"+destinations+trail
    
    #print(fullurl)
    response = requests.get(fullurl)
    response.raise_for_status()
    durations_matrix=json.loads(response.text)['durations']
    duraciones=[]
    destination_min=[]
    for origen in np.arange(np.shape(durations_matrix)[0]):
        durations+=[min(duraciones_matrix[origen])]
        destination_min+=[np.argmin(durations_matrix[origins])]
    for i in np.arange(len(durations)):
        origins.loc[[destinos_nombre[i]], ['t_'+origin_type]]=durations[i]/60./60.
        lugares.loc[[destinos_nombre[i]], ['m_'+origin_type]]=destination_min[i]